In [1]:
def drop_trials(epochs, chs=[0], do_peak=0, do_mean=0, do_slope=0, T1=0, T2=0, iter_inf=False):
    # Create a copy of epochs
    
    from scipy.stats import iqr
    from scipy.stats import median_abs_deviation as mad

    new_epochs = epochs.copy()
    # Create a list to keep track of trial status (1 for good trials, 0 for rejected trials)
    list_trials = np.ones(len(new_epochs))

    for j in chs:
        # Initialize variables for peak rejection
        peak_rejection = 0
        mean_rejection = 0
        slope_rejection = 0
        ite_peak = 0
        ite_mean = 0
        ite_slope = 0
        stop_peak = 0
        stop_mean = 0
        stop_slope = 0
        ite_loop = 0

        # Main loop for outlier detection and rejection
        while ite_mean > -1:
            ite_loop += 1

            if do_peak == 1:
                if ite_peak == 0:
                    # Calculate time indices based on T1 and T2 values
                    time1 = np.where(new_epochs.times < T1)[0][-1]
                    time2 = np.where(new_epochs.times < T2)[0][-1]

                    # Calculate peak-to-peak values for each trial and channel
                    peak_to_peak = np.array([max(new_epochs._data[i][j][time1:time2]) - min(new_epochs._data[i][j][time1:time2]) for i in range(len(new_epochs._data))], dtype=np.dtype(float))

                idx = []
                for i in range(len(new_epochs._data)):
                    value_prtp = peak_to_peak[i]

                    # Check if the value exceeds the upper threshold for peak rejection
                    outlier_up_peak = np.nanmedian(peak_to_peak) + (3 * mad(peak_to_peak, scale='normal', nan_policy='omit'))
                    if value_prtp > outlier_up_peak:
                        idx.append(i)

                if len(idx) == 0:
                    # No more outliers found, stop peak outlier detection
                    stop_peak = 1
                else:
                    ite_peak += 1
                    peak_to_peak[idx] = None
                    list_trials[idx] = 0
                    peak_rejection += len(idx)

            if do_slope == 1:
                if ite_slope == 0:
                    # Calculate time indices based on T1 and T2 values
                    time1 = np.where(new_epochs.times < T1)[0][-1]
                    time2 = np.where(new_epochs.times < T2)[0][-1]

                    x = new_epochs.times[time1:time2]
                    # Calculate slopes for each trial and channel
                    slopes_trials = np.array([np.polyfit(x, new_epochs._data[i, j, time1:time2], 1)[0] for i in range(len(new_epochs._data))], dtype=np.dtype(float))

                idx = []
                for i in range(len(new_epochs._data)):
                    value_prtp = slopes_trials[i]

                    # Check if the value exceeds the upper and lower thresholds for slope rejection
                    outlier_up_mean = np.nanmedian(slopes_trials) + (3 * mad(slopes_trials, scale='normal', nan_policy='omit'))
                    outlier_down_mean = np.nanmedian(slopes_trials) - (3 * mad(slopes_trials, scale='normal', nan_policy='omit'))
                    if value_prtp > outlier_up_mean:
                        idx.append(i)
                    elif value_prtp < outlier_down_mean:
                        idx.append(i)

                if len(idx) == 0:
                    # No more outliers found, stop slope outlier detection
                    stop_slope = 1
                else:
                    ite_slope += 1
                    slopes_trials[idx] = None
                    list_trials[idx] = 0
                    slope_rejection += len(idx)

            if do_mean == 1:
                if ite_mean == 0:
                    # Calculate time indices based on T1 and T2 values
                    time1 = np.where(new_epochs.times < T1)[0][-1]
                    time2 = np.where(new_epochs.times < T2)[0][-1]
                    # Calculate mean values for each trial and channel
                    mean_trials = np.array([np.mean(new_epochs._data[i][j][time1:time2]) for i in range(len(new_epochs._data))], dtype=np.dtype(float))

                idx = []
                for i in range(len(new_epochs._data)):
                    value_prtp = mean_trials[i]

                    # Check if the value exceeds the upper and lower thresholds for mean rejection
                    outlier_up_mean = np.nanmedian(mean_trials) + (3 * mad(mean_trials, scale='normal', nan_policy='omit'))
                    outlier_down_mean = np.nanmedian(mean_trials) - (3 * mad(mean_trials, scale='normal', nan_policy='omit'))
                    if value_prtp > outlier_up_mean:
                        idx.append(i)
                    elif value_prtp < outlier_down_mean:
                        idx.append(i)


                if len(idx) == 0:
                    # No more outliers found, stop mean outlier detection
                    stop_mean = 1
                else:
                    ite_mean += 1
                    mean_trials[idx] = None
                    list_trials[idx] = 0
                    mean_rejection += len(idx)

            # Check if any outlier detection is disabled
            if do_peak == 0 | iter_inf == False :
                stop_peak = 1
            if do_mean == 0 | iter_inf == False:
                stop_mean = 1
            if do_slope == 0 | iter_inf == False:
                stop_slope = 1

            # Check if all outlier detection has stopped
            if stop_mean == 1 and stop_peak == 1 and stop_slope == 1:
                # No more outlier detection, stop the loop
                break

    # Drop the trials marked for removal and return the modified epochs object
    new_epochs.drop(np.where(list_trials == 0)[0])
    return new_epochs, peak_rejection, mean_rejection, slope_rejection, list_trials, ite_loop


In [1]:
import os, mne, csv, re
import numpy as np
from glob import glob
from mne.epochs import equalize_epoch_counts

from mne.time_frequency import tfr_morlet
import matplotlib.pyplot as plt

#set directory pathway
os.chdir('C:/Users/mfbpe/Desktop/DATA/2022_deliberation/derivatives/')
data_files = glob('*ica_raw.fif')


In [3]:
%%capture
all=[]
participant = []
good_participant = []
drop_stats=[]
RT_part=[]
list_bad=[]
nb_delete={}
conditions = ['Arbi','Edeli','Hdeli']
# data_files_epochs = glob('*ica_hand_epo.fif')

datafile=open("data_RP.csv","w", newline="")
writer=csv.writer(datafile, delimiter=";")
writer.writerow(["Participant", "conditions","Trial_RP","outlier_RP","RT_RP", 'mean_RP_early','slope_RP_early', 'mean_RP_late','slope_RP_late', 'mean_RP','slope_RP'])

for part in range(len(data_files)):
    raw = mne.io.read_raw_fif(data_files[part],preload=True)
    n_part = int(re.split( "sub-|_2023",data_files[part])[1])
    
    if part == 0 :
        sphere = mne.make_sphere_model('auto', 'auto', raw.info)
        src = mne.setup_volume_source_space(sphere=sphere)
        forward = mne.make_forward_solution(raw.info, trans=None, src=src, bem=sphere)
    raw.set_eeg_reference('REST', forward=forward)

    events = mne.find_events(raw, shortest_event=1)
    ite_delete=0
    
    while events[0,2] not in [10,20,30] :
        events = np.delete(events,0,0)
        ite_delete+=1
    nb_delete[n_part]=ite_delete
    if part == 48:
        events[34,2]=20
    events = mne.merge_events(events, [13,14,15], 13)  # Beep Arbi
    events = mne.merge_events(events, [23,24,25], 23)  # Beep Edeli
    events = mne.merge_events(events, [33,34,35], 33)  # Beep Hdeli
    
    events = mne.merge_events(events, [11,12], 11)  # Hand Arbi
    events = mne.merge_events(events, [21,22], 21)  # Hand Edeli
    events = mne.merge_events(events, [31,32], 31)  # Hand Hdeli
                
                
    event_dict = {'Vis/Arbi':10,'Vis/Edeli':20,'Vis/Hdeli':30}

    # # try :
    vis= mne.Epochs(raw, events,baseline =None,event_id=event_dict,
            tmin=-1.1, tmax=0,preload=True, detrend=None, on_missing='ignore', picks='Cz')
     
    vis_events= mne.Epochs(raw, events,baseline =None,event_id=event_dict,
            tmin=-.1, tmax=0,preload=True, detrend=None, on_missing='ignore', picks='Cz') 

    event_dict = {'Hand/Arbi':11,'Hand/Edeli':21,'Hand/Hdeli':31}

    epochs= mne.Epochs(raw, events,baseline =None,event_id=event_dict,
            tmin=-3, tmax=1,preload=True, detrend=None, on_missing='ignore', picks='Cz') 

    hand = epochs['Hand']

    data = [round((hand.events[x,0]-vis_events.events[x,0])/512,2) for x in range(len(vis_events.events))]


    

    hand.shift_time(.1)
    hand.apply_baseline((-.05,.05))
    _,_,_,_,list_ti,_ = drop_trials(hand, do_mean=1, do_peak=1, do_slope=1, T1=-1,T2=0, chs=np.arange(len(epochs.ch_names)))
    drop_stats.append(sum(list_ti==0)/len(list_ti))

    time1_RP_early = np.where(hand.times<-1)[0][-1]
    time2_RP_early = np.where(hand.times<-.5)[0][-1]
    x_early = epochs.times[time1_RP_early:time2_RP_early]
    time1_RP_late = np.where(hand.times<-.5)[0][-1]
    time2_RP_late = np.where(hand.times<-0)[0][-1]
    x_late = epochs.times[time1_RP_late:time2_RP_late]
    x = epochs.times[time1_RP_early:time2_RP_late]
    hand._data *=1e6

    for ite_trial in range(len(hand)):
        RP_mean_early = np.mean(hand._data[ite_trial,0,time1_RP_early:time2_RP_early])
        RP_slope_early = np.polyfit(x_early,hand._data[ite_trial,0, time1_RP_early:time2_RP_early],1)[0]
        RP_mean_late = np.mean(hand._data[ite_trial,0,time1_RP_late:time2_RP_late])
        RP_slope_late = np.polyfit(x_late,hand._data[ite_trial,0, time1_RP_late:time2_RP_late],1)[0]
        RP_mean = np.mean(hand._data[ite_trial,0,time1_RP_early:time2_RP_late])
        RP_slope = np.polyfit(x,hand._data[ite_trial,0, time1_RP_early:time2_RP_late],1)[0]
        writer.writerow([n_part,  hand.events[ite_trial,2],ite_trial, list_ti[ite_trial],data[ite_trial],round(RP_mean_early,3), round(RP_slope_early,3),round(RP_mean_late,3), round(RP_slope_late,3),round(RP_mean,3), round(RP_slope,3)])

    hand.drop(np.where(list_ti==0)[0])
    hand.save(data_files[part].split('raw')[0]+'clean_rest_baseline005_02_03_2024_V2_epo.fif', overwrite=True)

    
datafile.close()


In [1]:
%%capture
import os, mne, csv
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

os.chdir('C:/Users/mfbpe/Desktop/DATA/2022_deliberation/derivatives/')
data_files = glob('*clean_rest_baseline005_02_03_2024_epo.fif')
data_files=np.delete(data_files,[10,23,25])
hand_all = [mne.read_epochs(x)  for x in data_files]

In [7]:
%%capture
gr = [x.apply_baseline((-.05,.05)).average() for x in hand_all]

In [6]:
%%capture
for  cond in ['Arbi','Edeli','Hdeli']:
    gr_all[cond] = [x[cond].apply_baseline((-0.05, 0.05)).crop(-1,.1).average(method="mean") for c,x in enumerate(hand_all) if c not in [30,31]]


In [7]:
import matplotlib
matplotlib.use("Qt5Agg")
mne.viz.plot_compare_evokeds(gr_all, ylim=dict(eeg=[-1.5e6, 3e6]),
                            linestyles=['solid', 'solid', 'solid'],  # Line styles for each grand average
                             styles={'Arbi': {"linewidth": 4},  # Style settings for the first grand average
                                     'Edeli': {"linewidth": 4},  # Style settings for the second grand average
                                     'Hdeli': {"linewidth": 4},  # Style settings for the third grand average
                                     },
                             
                             colors=["#f26d41","#40b2d8", "#ac4bd8"])

[<Figure size 1400x1050 with 2 Axes>]